In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from src.utils.helpers_sentiment_analysis import *

# Health topic

In [ ]:
# Health theme sentiment analysis
df_health = pd.read_csv('src/df_movies_themes/df_movies/movies_health.csv')
df_health = perform_sentiment_analysis(df_health)
df_health[['Decade', 'NLTK_Sentiment', 'TB_Sentiment', 'VADER_Sentiment', 'Emotions_Sentiment', 'Grouped_genres']]

In [ ]:
sentiment_counts = count_sentiments(df_health)
print(sentiment_counts)

We observe that there are more positive movies concerning health than negative ones.

In [ ]:
plot_movie_frequency_by_decade(df_health)

In [ ]:
plot_sentiment_by_decade(df_health, technique='NLTK')

In [ ]:
plot_sentiment_by_decade(df_health, technique='TextBlob')

In [ ]:
plot_sentiment_by_decade(df_health, technique='VADER')

In [ ]:
plot_sentiment_by_decade(df_health, technique='Emotions')

In [ ]:
plot_combined_sentiment_by_decade(df_health)

In [ ]:
plot_top_genres_overall(10, df_health)

In [ ]:
# [TODO] find better colors and fix the legend (display only the top 10 genres representated in graph)
plot_top_genres(10, df_health)